# extract data from sqllite


In [27]:
import sqlite3
import pandas as pd
import os

database_path = os.path.join(os.getcwd(), '../part1/1-do-by-phind/2-step2/0-KaggleData/CsharpLearningDatabase.sqlite')

# Now use this absolute path to connect
try:
    conn = sqlite3.connect(database_path)
except Exception as e:
    print(e)

# List all tables in the SQLite database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())


[('Repo',), ('sqlite_sequence',), ('Solution',), ('MethodParameter',), ('Method',)]


In [35]:
#query = "SELECT * FROM 'db_name' WHERE topic = 'C# Learning Data'"
#query = "SELECT * FROM 'Repo' WHERE 0=0"
#query = "SELECT * FROM 'sqlite_sequence' WHERE 0=0"
#query = "SELECT * FROM 'Solution' WHERE 0=0"
#query = "SELECT * FROM 'MethodParameter' WHERE 0=0"
query = "SELECT * FROM 'Method' WHERE 0=0"
df = pd.read_sql_query(query, conn)
# Use the data as needed

# Close the connection
conn.close()

## big cosShen?
### how should I train my model with this shit?

In [39]:
print(df.columns)

Index(['Id', 'Name', 'FullComment', 'Summary', 'ApiCalls', 'CommentIsXml',
       'SampledAt', 'SolutionId', 'Lang', 'NameTokenized'],
      dtype='object')


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'text' is the column with the text data and 'label' is the column with the labels
X = df['Name']
y = df['Summary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into numerical representations
vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Example of a simple LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_transformed.shape[1], 1), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_transformed, y_train, epochs=10, batch_size=32, validation_data=(X_test_transformed, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_transformed, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

In [ ]:
model.save('my_chatbot_model.h5')